# Extrator de trechos de texto V1.0 (preprocess 1/2)

Notebook para extracao de trechos de texto do DODF (proveniente do DODFminer) que contem as entidades anotadas manualmente.

Este notebook melhora a extracao comparado ao extract_text.ipynb, buscando trechos que comecam com o ATO (e.g. conceder aposentadoria) e terminam com o numero do processo. Alguns trechos de texto encontrados foram de atos tornados sem efeito (ao inves de concessao de aposentadoria), estes foram revisados e consertados manualmente.

In [123]:
import pandas as pd
import numpy as np
import unicodedata

In [124]:
# Carrega csv contendo as entidades nomeadas anotadas
verif_data = pd.read_csv('TCDF_data/Atos_Aposentadoria_validados.csv')

In [125]:
# Lista com datas dos DODFs analisados separados em ["dd", "mm", "aa", "idx"] e ordenadas em ordem crescente de data
l = []
for num in verif_data['DATA_DODF']:
#     if num not in l:
    l.append(num)

def func(a):
    return a[2], a[1], a[0]

for i in range(len(l)):
    l[i] = l[i].split('/')
    l[i][0] = int(l[i][0])
    l[i][1] = int(l[i][1])
    l[i][2] = int(l[i][2])
    l[i].append(i)
# n = np.argsort(np.array(l), axis=0)
n = l.copy()
l.sort(key = func)
print("Periodo de anotacao dos DODFs")
print("Data inicial:", f"{l[0][0]}/{l[0][1]}/{l[0][2]}")
print("Data final:", f"{l[-1][0]}/{l[-1][1]}/{l[-1][2]}")

Periodo de anotacao dos DODFs
Data inicial: 2/4/2018
Data final: 30/9/2019


In [126]:
import os
import re
month = ["01_Janeiro/", "02_Fevereiro/", "03_Março/", "04_Abril/", "05_Maio/", "06_Junho/", "07_Julho/", "08_Agosto/", "09_Setembro/", "10_Outubro/", "11_Novembro/", "12_Dezembro/"]
year = ["2018/", "2019/"]
path = 'dodfs/results/txt/'

aa = l[:2]

for date in aa:
    print(date[:3], date[3])
    print(verif_data.loc[date[3], 'NUM_DODF'], verif_data.loc[date[3], 'DATA_DODF'])
    p = path
    p += year[0] if date[2]==2018 else year[1]
    p += month[date[1]-1]
    files = os.listdir(p)
    split_files = [i.split() for i in files]

[2, 4, 2018] 963
62 02/04/2018
[2, 4, 2018] 964
62 02/04/2018


## Definindo regex

Regex sao utilizados para encontrar trechos de texto que contem as entidades nomeadas anotadas

In [127]:
# Regex para processo, nome e ato de uma anotacao
def get_regex(x, data):
    # regex de processo 
    aux = data.loc[x, 'PROCESSO']
    re_processo = ""
    for i in aux:
        re_processo += i + "[-\s]*"
    # regex de nome do servidor aposentado
    aux = data.loc[x, 'NOME_ATO']
    re_nome = ""
    for i in aux:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_nome += f"[{i.lower()}{i.upper()}]" + "[-,\s]*"
        else:
            re_nome += i + "[-\s]*"
    # regex de tipo (aposentadoria, conceder aposentadoria, etc) (NAO UTILIZADO)
    aux = data.iloc[x]['ATO']
    re_ato = ""
    for i in aux:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_ato += f"[{i.lower()}{i.upper()}]" + "[-,\s]*"
        else:
            re_ato += i + "[,-]*"
    # regex para tentar pegar ato completo (ato + nome + processo)
    re_full = re_ato + ".*?" + re_processo
    return re_full, re_ato, re_nome, re_processo

In [128]:
# Identifica um segmento de texto dadas as entidades da linha x do dataframe data
def get_segment(x, data):
    segment = None
    # Set bool to check if segment is found
    find = False
    # Get regex expression for this particular segment
    re_full, _, _, _ = get_regex(x, data)
    # Get list of DODF docs related to the num_DODF edition
    list_dodf = []
    num_dodf = data['NUM_DODF'][x]
    date_dodf = data['DATA_DODF'][x].split('/')
    path = 'dodfs/results/txt/'
    path += year[0] if int(date_dodf[2])==2018 else year[1]
    path += month[int(date_dodf[1])-1]
    docs = os.listdir(path)
    for f in docs:
        if int(f.split()[1]) == num_dodf:
            list_dodf.append(f)
    # Search each document in list_dodf
    for f in list_dodf:
        txt = open(path+"/"+f).read().replace('\n', ' ')
        segment = re.search(re_full, txt, flags=re.DOTALL)
        if segment:
            a = segment
            txt2 = txt
            while a:
                a = re.search(re_full, txt[segment.span()[0]+1:segment.span()[1]], flags=re.DOTALL)
                if a:
                    txt = txt[segment.span()[0]+1:segment.span()[1]]
                    segment = a
            find = True
            f = segment.span()
            segment = txt[f[0]:f[1]]
        if find:
            break
    return segment

# Itera sobre todas as linhas (rows) do dataframe, e chama get_segment se tem anotacao de PROCESSO nesta row
def get_text(data):
    for i in range(len(data)):
        if pd.isna(data['PROCESSO'][i]):
            continue
        else:
            seg = get_segment(i, data)
        if seg:
            data.loc[i, 'text'] = seg
    return data

In [129]:
verif_data['text'] = ""
verif_data = get_text(verif_data)
# verif_data.to_csv('preprocess_data1.csv', index=False)

## PARTE 2 -  Comeco do preprocessamento manual

Substituicao manual dos textos onde atos TORNAR SEM EFEITO foram obtidos ao inves de CONCEDER APOSENTADORIA (ou APOSENTAR)

In [130]:
print("Dados onde mencoes TORNAR SEM EFEITO foram obtidos ao inves de CONCEDER APOSENTADORIA")
cnt = 0
rows_to_fix = []
for row in range(len(verif_data)):
    if re.search("[Tt][-,]*[Oo][-,]*[Rr][-,]*[Nn][-,]*[aA][-,]*[rR] [Ss][-,]*[Ee][-,]*[Mm] [Ee][-,]*[fF][-,]*[Ee][-,]*[iI][-,]*[Tt][-,]*[Oo]", verif_data['text'][row]):
        print(f"Row:{row}    Data:{verif_data['DATA_DODF'][row]}    NUM_DODF:{verif_data['NUM_DODF'][row]}")
        rows_to_fix.append(row)
        cnt += 1
print("count:", cnt)

Dados onde mencoes TORNAR SEM EFEITO foram obtidos ao inves de CONCEDER APOSENTADORIA
Row:290    Data:04/05/2018    NUM_DODF:85
Row:594    Data:06/06/2018    NUM_DODF:106
Row:1125    Data:28/01/2019    NUM_DODF:19
Row:1147    Data:28/01/2019    NUM_DODF:19
Row:1182    Data:30/01/2019    NUM_DODF:21
Row:1875    Data:25/04/2019    NUM_DODF:77
Row:2905    Data:01/08/2019    NUM_DODF:144
Row:3032    Data:08/08/2019    NUM_DODF:149
Row:3033    Data:08/08/2019    NUM_DODF:149
Row:3064    Data:09/08/2019    NUM_DODF:150
Row:3846    Data:02/04/2018    NUM_DODF:62
Row:3993    Data:16/10/2018    NUM_DODF:197
Row:4261    Data:13/11/2018    NUM_DODF:216
Row:4416    Data:04/12/2018    NUM_DODF:81
Row:4419    Data:05/12/2018    NUM_DODF:230
Row:4420    Data:05/12/2018    NUM_DODF:230
Row:4434    Data:17/12/2018    NUM_DODF:238
Row:5376    Data:14/09/2018    NUM_DODF:176
count: 18


In [131]:
print("Rows to fix\n")

for row in rows_to_fix:
    print(f"Row:{row}")
    print(verif_data['ATO'][row], verif_data['NOME_ATO'][row], verif_data['PROCESSO'][row])
    print(f"DATA:{verif_data['DATA_DODF'][row]}  NUM_DODF:{verif_data['NUM_DODF'][row]}")
    print()

Rows to fix

Row:290
CONCEDER APOSENTADORIA MAURILIA ROSARIO SOARES DE CASTRO 080.003382/2017
DATA:04/05/2018  NUM_DODF:85

Row:594
CONCEDER APOSENTADORIA VILSON DE MATOS LIMA 00052.000.12548/2018-05
DATA:06/06/2018  NUM_DODF:106

Row:1125
CONCEDER APOSENTADORIA MARIA ADELIA NUNES FIGUEIREDO 468.000154/2011
DATA:28/01/2019  NUM_DODF:19

Row:1147
CONCEDER APOSENTADORIA SANDRA REGINA DOS SANTOS MENEZES 474.001039/2011
DATA:28/01/2019  NUM_DODF:19

Row:1182
CONCEDER APOSENTADORIA ALTAIR BEZERRA DE ARAUJO 00052-00000062/2019-05
DATA:30/01/2019  NUM_DODF:21

Row:1875
CONCEDER APOSENTADORIA GETULIO PEREIRA DOS SANTOS 276.001.828/2014
DATA:25/04/2019  NUM_DODF:77

Row:2905
CONCEDER APOSENTADORIA KARLA CRISTINA BORGES E SILVA AURORA 0008000178912/2018-35
DATA:01/08/2019  NUM_DODF:144

Row:3032
CONCEDER APOSENTADORIA DAVI MARQUES DA LUZ 270.002.004/2016
DATA:08/08/2019  NUM_DODF:149

Row:3033
CONCEDER APOSENTADORIA LUCIANO DE ALMEIDA FERRER 270.001.237/2016
DATA:08/08/2019  NUM_DODF:149

Row:30

In [132]:
verif_data.loc[290, 'text'] = "CONCEDER APOSENTADORIA a MAURILIA ROSARIO SOARES DE CASTRO, ma-tricula 175.338-X, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, doQuadro de Pessoal do Distrito Federal, nos termos do artigo 40,  1o, inciso III, alinea \"a\"e 3o, 5o,8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao dada pelaEmenda Constitucional no 41, de 31 de dezembro de 2003, e artigos 46 e 51 da LeiComplementar no 769, de 01 de julho de 2008. Processo 080.003382/2017."

verif_data.loc[594, 'text'] = "A DIRETORA DO DEPARTAMENTO DE GESTAO DE PESSOAS, DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o da Portaria no 3, de 11 de janeiro de 2012, e, ainda, o constante no processo 00052.000.12548/2018-05, RESOLVE: CONCEDER aposentadoria a VILSON DE MATOS LIMA, matricula no 39.491-2, no cargo efetivo de Perito Medico-Legista, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

verif_data.loc[1125, 'text'] = "CONCEDER APOSENTADORIA a MARIA ADELIA NUNES FIGUEIREDO matricula 53.765-9, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o, inciso I, II e III, e o paragrafo unico da Emenda Constitucional no 47, de 06 de julho de 2005, com as vantagens do artigo 5o da Lei no 4584, de 11 de julho de 2011. Processo 468.000154/2011."

verif_data.loc[1147, 'text'] = "CONCEDER APOSENTADORIA A SANDRA REGINA DOS SANTOS MENEZES, matricula 54.069-2, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 3o, inciso I, II e III, e paragrafo unico da Emenda Constitucional no 47, de 06 de julho de 2005, a contar de 10 de fevereiro de 2016. Processo no 4 7 4 . 0 0 1 0 3 9 / 2 0 11 ."

verif_data.loc[1182, 'text'] = "ALEXANDRE SERGIO VICENTE FERREIRA POLICIA CIVIL DO DISTRITO FEDERAL DEPARTAMENTO DE GESTAO DE PESSOAS PORTARIA DE 30 DE JANEIRO DE 2019 O DIRETOR DO DEPARTAMENTO DE GESTAO DE PESSOAS, DA POLICIA CIVIL DO DISTRITO FEDERAL, no uso de suas atribuicoes legais e tendo em vista a delegacao de competencia que lhe foi conferida pelo artigo 1o da Portaria no 3, de 11 de janeiro de 2012, e, ainda, o constante no processo 00052-00000062/2019-05, resolve: CONCEDER aposentadoria a ALTAIR BEZERRA DE ARAUJO, matricula no 57.053-2 no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

verif_data.loc[1875, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com o artigo 44 da Lei Complementar no 769, de 30/06/2008, a GETULIO PEREIRA DOS SANTOS, matricula 114.675-0, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - Tecnico Administrativo, Classe Especial - Padrao V, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal, a contar de 25/12/2017. Processo no 276.001.828/2014."

verif_data.loc[2905, 'text'] = "CONCEDER APOSENTADORIA a KARLA CRISTINA BORGES E SILVA AURORA, matricula 68.347-7, no Cargo de Agente de Gestao Educacional/Conservacao e Limpeza, Nivel 10, Padrao 3, Etapa V, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional n 47 de 06 de julho de 2005. Processo 00080- 00178912/2018-35."

verif_data.loc[3032, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a DAVI MARQUES DA LUZ, matricula n 141.049-0, na Carreira de Assistencia Publica a Saude, no cargo de Tecnico em Saude - Tecnico Administrativo, Primeira Classe, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal, a contar de 15/07/2019. Processo 270.002.004/2016."

verif_data.loc[3033, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, a LUCIANO DE ALMEIDA FERRER, matricula n 123.251-7, na Carreira Medica, no cargo MEDICO - ORTOPEDIA E TRAUMATOLOGIA, Classe Especial, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Processo 270.001.237/2016."

verif_data.loc[3064, 'text'] = "CONCEDER APOSENTADORIA a MARIA CRISTINA CARVALHO DE OLIVEIRA, matricula 35.847-9, no Cargo de Professor de Educacao Basica, Padrao 24, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o, incisos I, II, III e IV, da Emenda Constitucional no 41, de 31 de dezembro de 2003 e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00191372/2018-85."

verif_data.loc[3846, 'text'] = "O DIRETOR-GERAL DO DEPARTAMENTO DE ESTRADAS DE RODAGEM DO DIS- TRITO FEDERAL, usando das atribuicoes que lhe confere o artigo 106, inciso XXVI, do Regimento aprovado pelo Decreto no 37.949, de 12/01/2017 e com base na competencia delegada atraves do Decreto no 23.212, de 06/09/2002, RESOLVE: CONCEDER nos termos dos incisos I, II, III e paragrafo unico, artigo 3o, da Emenda Constitucional no 47, de 5 de julho de 2005, APOSENTADORIA VOLUNTARIA ao servidor ANTONIO EUSTAQUIO TAVARES, matricula no 93.867-X, Tecnico de Atividades Rodoviarias, Classe Especial, Padrao III, Referencia TR-S3, do Quadro de Pessoal do Departamento de Estradas de Rodagem do Distrito Federal. Processo no 00113-00007846/2018-40."

verif_data.loc[3993, 'text'] = "CONCEDER APOSENTADORIA a KESIA RAQUEL SILVA SANTANA, matricula 205.362-4, no Cargo de Professor de Educacao Basica, Padrao 24, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o, incisos I, II, III e IV, da Emenda Constitucional no 41, de 31 de dezembro de 2003 e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005, a contar de 03 de setembro de 2018. Processo 00080- 00060135/2018-73."

verif_data.loc[4261, 'text'] = "CONCEDER aposentadoria a FRANCISCO CARDOSO LOPES, matricula no 25.174-7, no cargo de Tecnico de Apoio as Atividades Policiais Civis, Classe Especial, Padrao III, da Carreira de Apoio as Atividades Policiais Civis, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e Paragrafo unico da Emenda Constitucional no 47 de 05/07/2005, a contar de 12/01/2013. Processo no 050.000.897/2011."

verif_data.loc[4416, 'text'] = "LUCIANA DA SILVA LIRA tituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41/2003, e artigos 18,  5o, 46 e 51 da Lei Complementar no 769/08, de acordo com o Laudo Medico no 042/2017 a CATARINA FERREIRA DE VASCONCELOS, matricula no 1.434.018-6, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - AUXILIAR DE ENFERMAGEM, Terceira Classe, Padrao VII, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: SRSLE. Processo: 00410- 00007736/2018-23."

verif_data.loc[4419, 'text'] = "APOSENTAR PETRUCIO RODRIGUES DA SILVA matricula 23.486-9, no Cargo de Profe s s o r, Classe A, Etapa/Referencia 25 BD -II, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40,  1o, inciso I, in fine, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012 e o artigo 18,  5o, da Lei Complementar no 769, de 01 de julho de 2008, a contar de 03 de marco de 2016. Processo 471.000056/2012."

verif_data.loc[4420, 'text'] = "CONCEDER APOSENTADORIA a HOSANO JORGE DAS NEVES matricula 48.541-1, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o da Emenda Constitucional no 47, de 06 de julho de 2005 e o paragrafo unico do mesmo artigo, com as vantagens previstas no artigo 5o da Lei no 4584, de 08 de julho de 2011, a contar de 23 de abril de 2017. Processo 080.010930/2016."

verif_data.loc[4434, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com o artigo 41,  7o da Lei Organica do Distrito Federal, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, a JOSE TADEU DOS SANTOS PALMIERI, matricula 122.332-1, na Carreira Medica no Cargo de Medico - Anestesiologia, Classe Especial, Padrao V, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal, a contar de 30/10/2015. Processo no 270.000.662/2013."

verif_data.loc[5376, 'text'] = "APOSENTAR o servidor MANOEL FERREIRA MATOS, matricula no 82.068-7, Agente de Gestao de Residuos Solidos, Classe Especial, Padrao VII, do Quadro de Pessoal do SLU, nos termos do artigo 40,  1o, inciso II, e  3o, 8o e 17, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 41, de 19/12/2003, e artigos 46 e 51 da Lei Complementar no 769, de 30/06/2008, a contar de 27/08/2012. Processo no 094.000.968/2012."

### Verificacao dos textos encontrados

Contagem do numero de trechos de texto identificados para as entidades anotadas ( 1 trecho de texto por linha do arquivo .csv lido)

In [133]:
# After preprocessing '\n'
empty_list = []
for i in range(len(verif_data)):
    if not verif_data['text'][i]:
        empty_list.append(i)
print("# de trechos ainda nao identificados:", len(empty_list))
empty_process = []
for i in range(len(verif_data)):
    if pd.isna(verif_data['PROCESSO'][i]):
        empty_process.append(i)
print("# de trechos nao identificados e sem anotacao de ['PROCESSO']:", len(empty_process))
print("# total de trechos no csv:", len(verif_data))

# de trechos ainda nao identificados: 347
# de trechos nao identificados e sem anotacao de ['PROCESSO']: 298
# total de trechos no csv: 5516


## PARTE 3 - Identificando trechos sem anotacao de ['PROCESSO']

Utilizando o fundamento legal para identificar os trechos de atos. Os que nao forem encontrados desta forma serao identificados manualmente.

### Utilizando fundamento legal para indentificar trechos sem ['PROCESSO']

In [134]:
# Regex para processo, nome e ato de uma anotacao
def get_regex_FUND(x, data):
    
    # regex de nome do servidor aposentado
    aux = data.loc[x, 'NOME_ATO']
    re_nome = ""
    for i in aux:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_nome += f"[{i.lower()}{i.upper()}]" + "[-,\s]*"
        else:
            re_nome += i + "[-\s]*"
            
    # regex de tipo (aposentadoria, conceder aposentadoria, etc) (NAO UTILIZADO)
    aux = data.loc[x, 'ATO']
    re_ato = ""
    for i in aux:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_ato += f"[{i.lower()}{i.upper()}]" + "[-,\s]*"
        else:
            re_ato += i + "[,-]*"
            
    # regex de fundamento legal
    aux = unicodedata.normalize('NFKD', data.loc[x, 'FUND_LEGAL']).encode('ascii', 'ignore').decode('utf8')
    re_fund = ""
    for i in aux:
        if (i >= 'a' and i <= 'z') or (i>='A' and i<='Z'):
            re_fund += f"[{i.lower()}{i.upper()}]" + "[,\-\s\"]*"
        else:
            re_fund += i + "[,\-\s\"]*"
    
    # regex para tentar pegar ato completo (ato + nome + processo)
    re_full = re_ato + ".*?" + re_nome + ".*?" + re_fund
    
    return re_full

# Identifica um segmento de texto dadas as entidades da linha x do dataframe data
def get_segment_FUND(x, data):
    segment = None
    # Set bool to check if segment is found
    find = False
    # Get regex expression for this particular segment
    re_full = get_regex_FUND(x, data)
    # Get list of DODF docs related to the num_DODF edition
    list_dodf = []
    num_dodf = data['NUM_DODF'][x]
    date_dodf = data['DATA_DODF'][x].split('/')
    path = 'dodfs/results/txt/'
    path += year[0] if int(date_dodf[2])==2018 else year[1]
    path += month[int(date_dodf[1])-1]
    docs = os.listdir(path)
    for f in docs:
        if int(f.split()[1]) == num_dodf:
            list_dodf.append(f)
    # Search each document in list_dodf
    for f in list_dodf:
        txt = open(path+"/"+f).read().replace('\n', ' ')
        segment = re.search(re_full, txt, flags=re.DOTALL)
        if segment:
            a = segment
            txt2 = txt
            while a:
                a = re.search(re_full, txt[segment.span()[0]+1:segment.span()[1]], flags=re.DOTALL)
                if a:
                    txt = txt[segment.span()[0]+1:segment.span()[1]]
                    segment = a
            find = True
            f = segment.span()
            segment = txt[f[0]:f[1]]
        if find:
            break
    return segment

In [135]:
from tqdm import tqdm

aux = verif_data.loc[(pd.isna(verif_data['PROCESSO']))]
not_found = []
for idx in tqdm(aux.index):
#     print("Index:", idx)
    txt = get_segment_FUND(idx, aux)
    if not txt:
        not_found.append(idx)
        
    verif_data.loc[idx, 'text'] = txt
len(aux), len(not_found)

100%|██████████| 298/298 [01:27<00:00,  3.42it/s]


(298, 79)

### Identificacao manual dos trechos sem ['PROCESSO']

In [136]:
# Fix row 348 | NOME: JOSE FRANCISCO MENDES DA GUIA | DATA: 09/05/2018 | NUM: 88
verif_data.loc[348, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, a JOSE FRANCISCO MENDES DA GUIA, matricula no 0132361X, na Carreira de Assistencia Publica a Saude, no Cargo de Auxiliar em Saude - ARTIFICE- OBRAS CIVIS, Classe Unica, Padrao XX, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC. Processo no 00060-00018575/2018-01"

# Fix row 1551 | NOME: MARIA APARECIDA ROCHA DE QUEIROZ | DATA: 15/03/2019 | NUM: 50
verif_data.loc[1551, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA APARECIDA ROCHA DE QUEIROZ, matricula 139.266-2, na Carreira de Assistencia Publica a Saude, no Cargo de Especialista em Saude - ASSISTENTE SOCIAL, Primeira Classe, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: GPCR. Processo: 0060-011633/2016"

# Fix row 2404 | NOME: MARIA APARECIDA MARQUES DOS SANTOS | DATA: 18/06/2019 | NUM: 114
verif_data.loc[2404, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a MARIA APARECIDA MARQUES DOS SANTOS, matricula no 141.004-0, na Carreira Medica, no Cargo de MEDICO - TERAPIA INTENSIVA ADULTO, Primeira Classe, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: NUCE/GPCR. Processo no 00060-00012834/2019-62"

# Fix row 3966 | NOME: FLAVIO JOSE MARTINS | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3966, 'text'] = "CONCEDER aposentadoria a FLAVIO JOSE MARTINS, matricula no 34.211-4, no carg o efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3969 | NOME: REGINA RODRIGUES DE OLIVEIRA ABREU | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3969, 'text'] = "CONCEDER aposentadoria a REGINA RODRIGUES DE OLIVEIRA ABREU, matricula no 58.022-8, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3971 | NOME: JOSE FRANKLIN COELHO DA SILVA FILHO | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3971, 'text'] = "CONCEDER aposentadoria a JOSE FRANKLIN COELHO DA SILVA FILHO, matricula no 39.593-5, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3972 | NOME: ELIZETE RODRIGUES DO LAGO | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3972, 'text'] = "CONCEDER aposentadoria a ELIZETE RODRIGUES DO LAGO, matricula no 57.511-9, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3973 | NOME: LUCIO RONALDO DE SOUSA | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3973, 'text'] = "CONCEDER aposentadoria a LUCIO RONALDO DE SOUSA, matricula no 35.894-0, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3974 | NOME: SERGIO LOPES REIS | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3974, 'text'] = "CONCEDER aposentadoria a SERGIO LOPES REIS, matricula no 35.895-9, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3976 | NOME: MARCONDES MORAES DE OLIVEIRA | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3976, 'text'] = "CONCEDER aposentadoria a MARCONDES MORAES DE OLIVEIRA, matricula no 37.574-8, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3977 | NOME: JULIO CESAR LUIS DA SILVA | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3977, 'text'] = "CONCEDER aposentadoria a JULIO CESAR LUIS DA SILVA, matricula no 45.136-3, no cargo efetivo de Perito Criminal, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 3986 | NOME: JASSE MARQUES DE MOURA | DATA: 16/10/2018 | NUM: 197
verif_data.loc[3986, 'text'] = "CONCEDER aposentadoria a JASSE MARQUES DE MOURA, matricula no 31.428-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4002 | NOME: REGIS PORTELA DA SILVA | DATA: 16/10/2018 | NUM: 197
verif_data.loc[4002, 'text'] = "CONCEDER aposentadoria a REGIS PORTELA DA SILVA, matricula no 34.167-3, no c a rg o efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4013 | NOME: JOSELY XAVIER DA ROCHA | DATA: 16/10/2018 | NUM: 197
verif_data.loc[4013, 'text'] = "CONCEDER aposentadoria a JOSELY XAVIER DA ROCHA, matricula no 31.637-7, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4021 | NOME: ADAO CESAR VALADAO DE FREITAS | DATA: 16/10/2018 | NUM: 197
verif_data.loc[4021, 'text'] = "CONCEDER aposentadoria a ADAO CESAR VALADAO DE FREITAS, matricula no 57.544-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4034 | NOME: LUCIO HENRIQUE DA SILVA FONSECA | DATA: 17/10/2018 | NUM: 198
verif_data.loc[4034, 'text'] = "CONCEDER APOSENTADORIA ESPECIAL, nos termos do artigo 40,  3o, 4o, inciso III, 8o e 17 da Constituicao da Republica Federativa do Brasil, com redacao da Emenda Cons- titucional no 41/2003, e no 47/2005, artigos 46 e 51 da Lei Complementar no 769/2008, e artigo 57 da Lei no 8213/1991, a LUCIO HENRIQUE DA SILVA FONSECA, matricula no 129261-7 , na Carreira Medica, no Cargo de Medico - ORTOPEDIA E TRAUMATOLOGIA, Classe Especial , Padrao IV , do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: ADMC Processo: 060.00114328/2018-26"

# Fix row 4041 | NOME: RAIMUNDA SONIA COSTA DA SILVA | DATA: 17/10/2018 | NUM: 198
verif_data.loc[4041, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a RAIMUNDA SONIA COSTA DA SILVA, matricula no 136.119-8, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - AUXILIAR DE ENFERMAGEM, Classe Especial, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: SRSLE. Processo: 00060-00212375/2018-34"

# Fix row 4054 | NOME: TANIA REJANE SILVA OLIVEIRA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4054, 'text'] = "CONCEDER APOSENTADORIA a TANIA REJANE SILVA OLIVEIRA, matricula 21.053- 6, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080.00027763/2018-47"

# Fix row 4056 | NOME: RAIMUNDA DE ARAUJO SOUSA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4056, 'text'] = "CONCEDER APOSENTADORIA a RAIMUNDA DE ARAUJO SOUSA, matricula 44.738- 2, no Cargo Agente de Gestao Educacional/Conservacao e Limpeza, Nivel 9, Padrao 3, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 3o, incisos I, II, III e paragrafo unico da Emenda Constitucional no 47 de 06 de julho de 2005. Processo 00080.00037496/2018-16"

# Fix row 4061 | NOME: MARIA HELENA LEITE | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4061, 'text'] = "CONCEDER APOSENTADORIA a MARIA HELENA LEITE, matricula 29.839-5, no Car- go de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Cons- titucional no 47, de 06 de julho de 2005. Processo 00080.00072077/2018-21"

# Fix row 4062 | NOME: MARIA HELENA FERREIRA DA SILVA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4062, 'text'] = "CONCEDER APOSENTADORIA a MARIA HELENA FERREIRA DA SILVA, matricula 67.514-8, no Cargo Agente de Gestao Educacional/Servicos Gerais, Nivel 10, Padrao 2, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 3o, incisos I, II, III e paragrafo unico da Emenda Constitucional no 47 de 06 de julho de 2005. Processo 00080.00084588/2018-95"

# Fix row 4063 | NOME: MARIA DIVINA FERREIRA DOS SANTOS | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4063, 'text'] = "CONCEDER APOSENTADORIA a MARIA DIVINA FERREIRA DOS SANTOS, ma- tricula 49.486-0, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080.00028881/2018-72"

# Fix row 4064 | NOME: JACIARA ALVES VAZ | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4064, 'text'] = "CONCEDER APOSENTADORIA a JACIARA ALVES VAZ, matricula 64.931-7, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Cons- titucional no 47, de 06 de julho de 2005. Processo 00080.00085736/2018-99"

# Fix row 4065 | NOME: HELIO MIGUEL DA SILVEIRA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4065, 'text'] = "CONCEDER APOSENTADORIA a HELIO MIGUEL DA SILVEIRA, matricula 52.938-9, no Cargo Tecnico de Gestao Educacional/Conducao de Veiculos, Nivel 11, Padrao 1, Etapa V, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 3o, incisos I, II, III e paragrafo unico da Emenda Constitucional no 47 de 06 de julho de 2005, com as vantagens previstas no artigo 5o da Lei no 4.584, de 08 de julho de 2011. Processo 00080.00026012/2017-22"

# Fix row 4067 | NOME: SILVANA DA SILVA TEIXEIRA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4067, 'text'] = "CONCEDER APOSENTADORIA a SILVANA DA SILVA TEIXEIRA, matricula 26.366-4 no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080.00095409/2018-45"

# Fix row 4070 | NOME: CLAUDIO TELLES FERREIRA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4070, 'text'] = "APOSENTAR CLAUDIO TELLES FERREIRA, matricula 27.591-3 no Cargo de Professor de Educacao Basica, Padrao 24, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, in fine, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012, e o artigo 18,  5o, da Lei Complementar no 769, de 01 de julho de 2008. Processo 00410-00010285/2018-10"

# Fix row 4072 | NOME: MARCIO ANTONIO SOUSA DA SILVA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4072, 'text'] = "APOSENTAR MARCIO ANTONIO SOUSA DA SILVA, matricula 201.326-6, no Cargo de Professor de Educacao Basica, Padrao 20, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, in fine, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012, e o artigo 18,  5o, da Lei Complementar no 769, de 01 de julho de 2008. Processo 00410-00009598/2018- 17"

# Fix row 4074 | NOME: KARLA CARRARA DE OLIVEIRA ALCANTARA | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4074, 'text'] = "APOSENTAR KARLA CARRARA DE OLIVEIRA ALCANTARA, matricula 25.773-7, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00410-00009603/2018-91"

# Fix row 4075 | NOME: CYNTHIA MARTINS MACHADO | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4075, 'text'] = "APOSENTAR CYNTHIA MARTINS MACHADO, matricula 32.951-7, no Cargo de Pro- fessor de Educacao Basica, Padrao 23, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00410-00010296/2018- 91"

# Fix row 4078 | NOME: ADILCEIA MARIA BETONICO | DATA: 22/10/2018 | NUM: 201
verif_data.loc[4078, 'text'] = "APOSENTAR ADILCEIA MARIA BETONICO, matricula 200.219-1, no Cargo de Pro- fessor de Educacao Basica, Padrao 22, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00410-00011 0 7 7 / 2 0 1 8 - 20"

# Fix row 4079 | NOME: EDSON DIAS NUNES | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4079, 'text'] = "CONCEDER aposentadoria a EDSON DIAS NUNES, matricula no 48.097-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4080 | NOME: CARLOS ALBERTO ELIAS DE SOUZA | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4080, 'text'] = "CONCEDER aposentadoria a CARLOS ALBERTO ELIAS DE SOUZA, matricula no 36.586-6, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4081 | NOME: MARCORY GERALDO MOHN | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4081, 'text'] = "CONCEDER aposentadoria a MARCORY GERALDO MOHN, matricula no 25.451-7, no cargo efetivo de Delegado de Policia, Classe Especial, da Carreira de Delegado de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4082 | NOME: JOSE ADALBERTO ALVES DO AMARAL | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4082, 'text'] = "CONCEDER aposentadoria a JOSE ADALBERTO ALVES DO AMARAL, matricula no 27.704-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4083 | NOME: CESAR MOREIRA DA SILVA | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4083, 'text'] = "CONCEDER aposentadoria a CESAR MOREIRA DA SILVA, matricula no 39.366-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4084 | NOME: SANDRA REGINA ROLIM PINHEIRO RESENDE | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4084, 'text'] = "CONCEDER aposentadoria a SANDRA REGINA ROLIM PINHEIRO RESENDE, ma- tricula no 39.497-1, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Cons- titucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4085 | NOME: JOSE VALTO CARLOS SOUZA | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4085, 'text'] = "CONCEDER aposentadoria a JOSE VALTO CARLOS SOUZA, matricula no 47.830-X, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4086 | NOME: JOSE HENRIQUE PEIXOTO DE BARROS | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4086, 'text'] = "CONCEDER aposentadoria a JOSE HENRIQUE PEIXOTO DE BARROS, matricula no 35.207-1, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4087 | NOME: MARCO ANTONIO MORENO CARDOSO | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4087, 'text'] = "CONCEDER aposentadoria a MARCO ANTONIO MORENO CARDOSO, matricula no 46.880-0, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4088 | NOME: EDSON DE JESUS LIMA | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4088, 'text'] = "CONCEDER aposentadoria a EDSON DE JESUS LIMA, matricula no 27.885-8, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4089 | NOME: CLAUDIO DE LIMA MILITAO | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4089, 'text'] = "CONCEDER aposentadoria a CLAUDIO DE LIMA MILITAO, matricula no 31.377-7, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4090 | NOME: JURANDINA ALVES DOS SANTOS | DATA: 30/10/2018 | NUM: 207
verif_data.loc[4090, 'text'] = "CONCEDER aposentadoria a JURANDINA ALVES DOS SANTOS, matricula no 35.817-7, no cargo efetivo de Papiloscopista Policial, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4113 | NOME: LOURDEFATIMA TEIXEIRA NOGUEIRA | DATA: 31/10/2018 | NUM: 208
verif_data.loc[4113, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 6o da Emenda Constitucional no 41/2003, combinado com o artigo 2o da Emenda Constitucional no 47/2005, combinados com o artigo 43, da Lei Complementar no 769, de 30/06/2008, a LOURDEFATIMA TEIXEIRA NOGUEIRA, matricula no 156.725X, na Carreira de ENFERMEIRO, no Cargo de EN- FERMEIRO, Segunda Classe, Padrao IV Segunda Classe, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: SRSSU. Processo: 00060- 00405332/2018-09"

# Fix row 4181 | NOME: ACHILLES BENEDITO DE OLIVEIRA | DATA: 05/11/2018 | NUM: 210
verif_data.loc[4181, 'text'] = "CONCEDER aposentadoria a ACHILLES BENEDITO DE OLIVEIRA, matricula no 25.446- 0, no cargo efetivo de Delegado de Policia, Classe Especial, da Carreira de Delegado de Policia do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Com- plementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Cons- titucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4247 | NOME: LINDALVA MATOS RIBEIRO FARIAS | DATA: 09/11/2018 | NUM: 214
verif_data.loc[4247, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com o artigo 41,  7o da Lei Organica do Distrito Federal, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, a LINDALVA MATOS RIBEIRO FARIAS, matricula no 137627-6 na Carreira de ENFERMEIRO, no Cargo de ENFERMEIRA, Classe Especial, Padrao II, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: FEPECS. Processo: 00064-0000.1585/2018-96"

# Fix row 4254 | NOME: VALMISA PEREIRA GOMES | DATA: 09/11/2018 | NUM: 214
verif_data.loc[4254, 'text'] = "CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, a VALMISA PEREIRA GOMES matricula no 124225-3, na Carreira de Assistencia Publica a Saude, no Cargo de Tecnico em Saude - AUXILIAR DE ENFERMAGEM, Classe Especial, Padrao V, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: DHMIB. Processo: 00060.00028644/2018-86"

# Fix row 4265 | NOME: EDUARDO FELIPE DAHER | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4265, 'text'] = "CONCEDER aposentadoria a EDUARDO FELIPE DAHER, matricula no 39.898-5, no cargo efetivo de Perito Criminal, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41/2003, c/c o artigo 2o da Emenda Constitucional no 41/2003."

# Fix row 4266 | NOME: DAURI BATISTA | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4266, 'text'] = "CONCEDER aposentadoria a DAURI BATISTA, matricula no 47.268-9, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4267 | NOME: ANDIARA MARIA ARAUJO SAMPAIO | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4267, 'text'] = "CONCEDER aposentadoria a ANDIARA MARIA ARAUJO SAMPAIO, matricula no 48.065-7, no cargo efetivo de Papiloscopista Policial, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4268 | NOME: WALTER COSTA VERGNIAUD | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4268, 'text'] = "CONCEDER aposentadoria a WALTER COSTA VERGNIAUD, matricula no 76.397-7, no cargo efetivo de Delegado de Policia, Classe Especial, da Carreira de Delegado de Policia do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4269 | NOME: DENISE LARA DOS SANTOS | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4269, 'text'] = "CONCEDER aposentadoria a DENISE LARA DOS SANTOS, matricula no 58.305-7, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4270 | NOME: CRISTIANNE MARIA FAULHABER LOPES | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4270, 'text'] = "CONCEDER aposentadoria a CRISTIANNE MARIA FAULHABER LOPES, matricula no 58.477-0, no cargo efetivo de Agente Policial de Custodia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4271 | NOME: HORTENIZIO DOMINGOS DE MESQUITA | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4271, 'text'] = "CONCEDER aposentadoria a HORTENIZIO DOMINGOS DE MESQUITA, matricula no 47.568-8, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4272 | NOME: RONALDO DIAS CORREA | DATA: 16/11/2018 | NUM: 218
verif_data.loc[4272, 'text'] = "CONCEDER aposentadoria a RONALDO DIAS CORREA, matricula no 172.094-5, no cargo efetivo de Agente Policial de Custodia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4330 | NOME: VANDA GONCALVES DE CARVALHO | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4330, 'text'] = "CONCEDER aposentadoria a VANDA GONCALVES DE CARVALHO, matricula no 37.967-0, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4331 | NOME: PAULO VILARINS DOS SANTOS | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4331, 'text'] = "CONCEDER aposentadoria a PAULO VILARINS DOS SANTOS, matricula no 40.053-X, no cargo efetivo de Perito Criminal, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4332 | NOME: GUILHERME ROCHA DE ALMEIDA ABREU | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4332, 'text'] = "CONCEDER aposentadoria a GUILHERME ROCHA DE ALMEIDA ABREU, matricula no 39.337-1, no cargo efetivo de Perito Criminal, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4333 | NOME: CARLOS ALBERTO RODRIGUES TABANEZ | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4333, 'text'] = "CONCEDER aposentadoria a CARLOS ALBERTO RODRIGUES TABANEZ, matricula no 47.158-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4334 | NOME: FRANCISCO FERNANDES DE FREITAS | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4334, 'text'] = "CONCEDER aposentadoria a FRANCISCO FERNANDES DE FREITAS, matricula no 57.028-1, no cargo efetivo de Papiloscopista Policial, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4335 | NOME: DANIELA VITORINO DA SILVA ROCHA | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4335, 'text'] = "CONCEDER aposentadoria a DANIELA VITORINO DA SILVA ROCHA, matricula no 57.628-X, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4336 | NOME: CLAUDIO CESAR RODRIGUES PEREIRA | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4336, 'text'] = "CONCEDER aposentadoria a CLAUDIO CESAR RODRIGUES PEREIRA, matricula no 219.637-9, no cargo efetivo de Agente Policial de Custodia, Primeira Classe, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4337 | NOME: RUBENS DE ARAUJO | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4337, 'text'] = "CONCEDER aposentadoria a RUBENS DE ARAUJO, matricula no 47.447-9, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4338 | NOME: FABIO TEIXEIRA LEMOS | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4338, 'text'] = "CONCEDER aposentadoria a FABIO TEIXEIRA LEMOS, matricula no 23.456-7, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4339 | NOME: PAULO CESAR DA COSTA | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4339, 'text'] = "CONCEDER aposentadoria a PAULO CESAR DA COSTA, matricula no 39.449-1, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4340 | NOME: MARIA DO CARMO CORREIA | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4340, 'text'] = "CONCEDER aposentadoria a MARIA DO CARMO CORREIA, matricula no 57.413-9, no cargo efetivo de Delegado de Policia, Classe Especial, da Carreira de Delegado de Policia do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4341 | NOME: CARLOS ROBERTO DA CUNHA MELO | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4341, 'text'] = "CONCEDER aposentadoria a CARLOS ROBERTO DA CUNHA MELO, matricula no 31.412-9, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4342 | NOME: FERNANDES HENRIQUE GONCALVES | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4342, 'text'] = "CONCEDER aposentadoria a FERNANDES HENRIQUE GONCALVES, matricula no 35.889-4, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4343 | NOME: MAURICIO RODRIGUES GUIMARAES | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4343, 'text'] = "CONCEDER aposentadoria a MAURICIO RODRIGUES GUIMARAES, matricula no 35.785-5, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4344 | NOME: ELVIS ADRIANO DA SILVA OLIVEIRA | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4344, 'text'] = "CONCEDER aposentadoria a ELVIS ADRIANO DA SILVA OLIVEIRA, matricula no 37.887-9, no cargo efetivo Perito Medico-Legista, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4345 | NOME: SILVINHO DE SOUSA SANTOS | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4345, 'text'] = "CONCEDER aposentadoria a SILVINHO DE SOUSA SANTOS, matricula no 35.171-7, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003."

# Fix row 4346 | NOME: SAMUEL MACEDO SILVA | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4346, 'text'] = 'CONCEDER aposentadoria a SAMUEL MACEDO SILVA, matricula no 31.622-9, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4347 | NOME: ERIC SEBA DE CASTRO | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4347, 'text'] = 'CONCEDER aposentadoria a ERIC SEBA DE CASTRO, matricula no 25.536-X, no cargo efetivo de Delegado de Policia, Classe Especial, da Carreira de Delegado de Policia do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4348 | NOME: ADOVALDO PEREIRA DE OLIVEIRA FILHO | DATA: 27/11/2018 | NUM: 225
verif_data.loc[4348, 'text'] = 'CONCEDER aposentadoria a ADOVALDO PEREIRA DE OLIVEIRA FILHO, matricula no 31.824-8, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4431 | NOME: RAPHAEL NASSER DE OLIVEIRA | DATA: 17/12/2018 | NUM: 238
verif_data.loc[4431, 'text'] = 'APOSENTAR RAPHAEL NASSER DE OLIVEIRA, matricula no 58.355-3, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 40,  1o, inciso I, da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, na redacao dada pela Emenda Constitucional no 41/2003, combinado com o artigo 6-A da Emenda Constitucional no 41/03, incluido pela EC no 70/2012, e artigos 186, inciso I, in fine e 189, paragrafo unico da Lei 8.112, de 11/12/90.'

# Fix row 4432 | NOME: JOSE GERALDO ALVES DA SILVA | DATA: 17/12/2018 | NUM: 238
verif_data.loc[4432, 'text'] = 'CONCEDER aposentadoria a JOSE GERALDO ALVES DA SILVA, matricula no 47.198-4, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4433 | NOME: AVSK NUREYEV SILVEIRA BUENO | DATA: 17/12/2018 | NUM: 238
verif_data.loc[4433, 'text'] = 'CONCEDER aposentadoria a AVSK NUREYEV SILVEIRA BUENO, matricula no 57.693-X, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4465 | NOME: JORGE LUIZ DA COSTA SANTOS | DATA: 26/12/2018 | NUM: 244
verif_data.loc[4465, 'text'] = 'CONCEDER aposentadoria a JORGE LUIZ DA COSTA SANTOS, matricula no 57.122-9, no cargo efetivo de Escrivao de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4466 | NOME: DANYELLE DA SILVA FEITOSA | DATA: 26/12/2018 | NUM: 244
verif_data.loc[4466, 'text'] = 'APOSENTAR DANYELLE DA SILVA FEITOSA, matricula no 231.519-X, no cargo efetivo de Agente de Policia, Terceira Classe, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 40,  1o, inciso I, da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, na redacao dada pela Emenda Constitucional no 41/2003, combinado com o artigo 6-A da Emenda Constitucional no 41/03, incluido pela EC no 70/2012, e artigos 186, inciso I, in fine, e 190 da Lei 8.112, de 11/12/90.'

# Fix row 4467 | NOME: LUCAS BRANDAO BRANCO DE SOUZA | DATA: 26/12/2018 | NUM: 244
verif_data.loc[4467, 'text'] = 'APOSENTAR LUCAS BRANDAO BRANCO DE SOUZA, matricula no 217.014-0, no cargo efetivo de Agente de Policia, Segunda Classe, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 40,  1o, inciso I, 3o, 8o e 17 da CONSTITUICAO DA REPUBLICA FEDERATIVA DO BRASIL, na redacao dada pela Emenda Constitucional no 41/2003, e artigos 1o e 15 da Lei no 10.887/2004.'

## PARTE 4 - Identificando manualmente os trechos ainda nao identificados

In [137]:
not_found = []
for idx in verif_data.index:
    if verif_data.loc[idx, 'text']=="":
        not_found.append(idx)
len(not_found), len(verif_data)

(49, 5516)

In [138]:
# Fix row 55 | NOME: NINA MARIA DA SILVA NEVES GADELHA | DATA: 07/01/2019 | NUM: 4
verif_data.loc[55, 'text'] = 'MARCIO MICHEL PORTARIA No 8, DE 03 DE JANEIRO DE 2019 O PRESIDENTE EM EXERCICIO DO TRIBUNAL DE CONTAS DO DISTRITO FEDERAL, no uso da atribuicao que lhe confere o art. 68, inc. III, da Lei Complementar no 1, de 9 de maio de 1994, e tendo em vista o que se apresenta no Processo no 38.613/2018-e, resolve: CONCEDER aposentadoria voluntaria a servidora NINA MARIA DA SILVA NEVES GADELHA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 914-8, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 116 | NOME: CLAUDIA CRISTIANE CAVALCANTE AIELLO | DATA: 10/04/2018 | NUM: 68
verif_data.loc[116, 'text'] = 'CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: CLAUDIA CRISTIANE CAVA L - CANTE AIELLO, matricula 100.723-8, processo SEI n 00070.00011832/2018-38, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008.'

# Fix row 124 | NOME: WILSON FRANCISCO DA SILVA | DATA: 19/04/2018 | NUM: 75
verif_data.loc[124, 'text'] = 'CONCEDER apo- sentadoria a WILSON FRANCISCO DA SILVA, matricula no 58.678-1, no cargo efetivo de Agente Policial de Custodia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 6o da Emenda Constitucional no 41/2003 c/c o artigo 2o da Emenda Constitucional no 47/2005.'

# Fix row 233 | NOME: ANTONIO BARBOZA DE SOUZA | DATA: 02/05/2018 | NUM: 83
verif_data.loc[233, 'text'] = 'CONCEDER aposentadoria voluntaria com proventos integrais ao servidor a seguir nominado: ANTONIO BARBOZA DE SOUZA, matricula 0100837-4, processo SEI n 00070-00012020/2018-18, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pes- soal do Distrito Federal, com fundamento no Art. 6, incisos I, II, III e IV, da EC n 41/ 2003, combinado com o artigo 2 da EC n 47/2005, c/c a Lei Complementar no 769/08.'

# Fix row 236 | NOME: FATIMA CONCEICAO ALDAVE MARTINS | DATA: 02/05/2018 | NUM: 83
verif_data.loc[236, 'text'] = 'CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: FATIMA CONCEICAO ALD AV E MARTINS, matricula 100.997-4, processo SEI n 00070-00012756/2018-88 , no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 6, incisos I, II, III e IV, da EC n 41/ 2003, combinado com o artigo 2 da EC n 47/2005, c/c a Lei Complementar no 769/08.'

# Fix row 240 | NOME: ANA ALVES DA SILVA | DATA: 02/05/2018 | NUM: 83
verif_data.loc[240, 'text'] = 'CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: ANA ALVES DA SILVA, matricula 100.822-6, processo SEI n 00070- 00012467/2018-89, no cargo de Auxiliar de Desen- volvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008.'

# Fix row 241 | NOME: SOLAZA CRISTINA CAETANO DOS SANTOS DE SANT'ANA | DATA: 02/05/2018 | NUM: 83
verif_data.loc[241, 'text'] = "CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: SOLAZA CRISTINA CAETA N O DOS SANTOS DE SANT\'ANA, matricula 100.817-X, processo SEI n 00070- 000112576/2018-04, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pessoal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008."

# Fix row 403 | NOME: JOAO ALBERTO GOMES MACHADO | DATA: 15/05/2018 | NUM: 92
verif_data.loc[403, 'text'] = 'CONCEDER nos termos dos incisos I, II, III e paragrafo unico, artigo 3o, da Emenda Constitucional no 47, de 5 de julho de 2005, APOSENTADORIA VOLUNTARIA ao ser- vidor JOAO ALBERTO GOMES MACHADO, matricula no 93.464-X, Tecnico de Ati- vidades Rodoviarias, Classe Especial, Padrao III, Referencia TR-S3, do Quadro de Pessoal do Departamento de Estradas de Rodagem do Distrito Federal. Processo no 00113- 00014384/2018-17.'

# Fix row 438 | NOME: IVANI SILVA FERREIRA | DATA: 18/05/2018 | NUM: 95
verif_data.loc[438, 'text'] = 'CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, bem como as vantagens de gratificacao de Raio-X, de que trata a Lei no 1.234/1950, em consonancia com o artigo 34,  2o da Lei no 4.345/1964, alterada pela Lei no 6.786/1980, combinado com o Decreto-lei no 1.883/1981 a IVANI SILVA FERREIRA, matricula no 125.010-8, na Carreira de Assistencia Publica a Saude no cargo de Tecnico em Saude - Tecnico em Radiologia, Classe Especial, Padrao V, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: SRSSO. Processo: 284.000433/2016(SEI).'

# Fix row 507 | NOME: CLEUSA DE ALCANTARA | DATA: 04/06/2018 | NUM: 104
verif_data.loc[507, 'text'] = 'CONCEDER aposentadoria voluntaria com proventos integrais a servidora a seguir nominada: CLEUSA DE ALCANTARA, ma- tricula 100.844-7, processo SEI n 00070- 00012862/2018-61, no cargo de Auxiliar de Desenvolvimento e Fiscalizacao Agropecuaria, Classe Unica, Padrao X, do Quadro de Pes- soal do Distrito Federal, com fundamento no Art. 3, incisos I, II, III e paragrafo unico da EC no 47/2005, c/c a LC no 769/2008.'

# Fix row 965 | NOME: MARIA EMILIA GONCALVES DA SILVA | DATA: 02/04/2018 | NUM: 62
verif_data.loc[965, 'text'] = 'CONCEDER, aposentadoria voluntaria integral, a servidora MARIA EMILIA GONCALVES DA SILVA, matricula n 23.770-1, no cargo de Analista de Politicas Publicas e Gestao Governamental, Classe Especial, Padrao V do Quadro de Pessoal do Distrito Federal, nos termos do artigo 6, incisos, I, II, III e IV da Emenda Constitucional n 41, de 19.02.2003, com redacao dada pelo artigo 2 da Emenda Constitucional n 47 de 05/07/2005, combinado com o artigo 43 da Lei Complementar n 769, de 30/06/2008, com a vantagem pessoal prevista no artigo 5 da Lei n 4.584, 08.07.2011. Processo SEI n 0 0 0 5 0 . 0 0 0 11 4 1 3 / 2 0 1 8 - 7 1 .'

# Fix row 973 | NOME: IVONETE PETRONILO DE AQUINO | DATA: 02/04/2018 | NUM: 62
verif_data.loc[973, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora IVONETE PETRONILO DE AQUINO, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula 988-1, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 1o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 982 | NOME: JOSE ISMAEL DOS SANTOS | DATA: 04/04/2018 | NUM: 64
verif_data.loc[982, 'text'] = 'CONCEDER nos termos do artigo 6o, incisos I, II, III e IV, da Emenda Constitucional no 41, de 19 de dezembro de 2003, combinado com o artigo 2o, da Emenda Constitucional no 47, de 05 de julho de 2005, APOSENTADORIA VOLUNTARIA ao servidor JOSE ISMAEL DOS SANTOS, matricula no 94.067-4, Tecnico de Atividades Rodoviarias, Classe Especial, Padrao III, Referencia TR- S3, do Quadro de Pessoal do Departamento de Estradas de Rodagem do Distrito Federal. Processo no 113.00008493/2018-03.'

# Fix row 1059 | NOME: EDUARDO MARIA DE MACEDO FRANCA | DATA: 15/01/2019 | NUM: 10
verif_data.loc[1059, 'text'] = 'Conceder aposentadoria voluntaria ao servidor EDUARDO MARIA DE MACEDO FRANCA, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula 276-3, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 1070 | NOME: LEONIDAS PEREIRA DE ALMEIDA | DATA: 21/01/2019 | NUM: 14
verif_data.loc[1070, 'text'] = 'CONCEDER aposentadoria a LEONIDAS PEREIRA DE ALMEIDA, matricula no 57.863-0, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea "a", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA REPUBLICA.'

# Fix row 1227 | NOME: SANDRA DE ANDRADE PACHECO | DATA: 18/02/2019 | NUM: 34
verif_data.loc[1227, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora SANDRA DE ANDRADE PACHECO, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula no 1270-0, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1261 | NOME: JOSEFA DA SILVA RIBEIRO DE AVILA | DATA: 04/02/2019 | NUM: 24
verif_data.loc[1261, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora JOSEFA DA SILVA RIBEIRO DE AVILA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula no 1021-9, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 1398 | NOME: FRANCISCO FIDELMAN ALVES DE LIMA | DATA: 26/02/2019 | NUM: 16
verif_data.loc[1398, 'text'] = 'CONCEDER aposentadoria a FRANCISCO FIDELMAN ALVES DE LIMA, matricula no 46.846-0, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"a\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da CONSTITUICAO DA RE- PUBLICA FEDERATIVA DO BRASIL, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 1419 | NOME: JOAO ADRIANO DE CARVALHO | DATA: 01/03/2019 | NUM: 43
verif_data.loc[1419, 'text'] = 'Conceder aposentadoria voluntaria ao servidor JOAO ADRIANO DE CARVALHO, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula 500-2, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1606 | NOME: EDNALDO NOGUEIRA AMARAL | DATA: 25/03/2019 | NUM: 56
verif_data.loc[1606, 'text'] = 'Conceder aposentadoria voluntaria ao servidor EDNALDO NOGUEIRA AMARAL, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 1199-1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1607 | NOME: ITAMAR LINO DE OLIVEIRA | DATA: 28/03/2019 | NUM: 59
verif_data.loc[1607, 'text'] = 'Conceder aposentadoria voluntaria ao servidor ITAMAR LINO DE OLIVEIRA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 1101- 1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 1611 | NOME: CLERIO BORBA BRASIL | DATA: 01/04/2019 | NUM: 61
verif_data.loc[1611, 'text'] = 'CONCEDER aposentadoria voluntaria ao servidor CLERIO BORBA BRASIL, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula no 414-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 1665 | NOME: ALEXANDRE ROZENWALD | DATA: 02/04/2019 | NUM: 62
verif_data.loc[1665, 'text'] = 'CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, bem como as vantagens do art. 5 da Lei 4.584/2011, a ALEXANDRE ROZENWALD, matricula no 128.377-4, na Carreira Medica, no cargo MEDICO - GENERALISTA, Classe Especial, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: CRDF. Processo SEI n 0060-009670/2013.'

# Fix row 1716 | NOME: LUCIANA DE ALBUQUERQUE MELLO SEIXAS | DATA: 05/04/2019 | NUM: 65
verif_data.loc[1716, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora LUCIANA DE ALBUQUERQUE MELLO SEIXAS, Auditora de Controle Externo, Classe Especial, Padrao 63, matricula no 546-1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2000 | NOME: ROSELI RAPOSO | DATA: 08/05/2019 | NUM: 85
verif_data.loc[2000, 'text'] = 'Conceder aposentadoria voluntaria a servidora ROSELI RAPOSO, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula 1030-8, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2101 | NOME: JOSIMAR DIAS MATOS | DATA: 22/05/2019 | NUM: 95
verif_data.loc[2101, 'text'] = 'CONCEDER aposentadoria voluntaria a JOSIMAR DIAS MATOS, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 836-2, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2272 | NOME: GIVALDO ANTONIO BATISTA DA CUNHA | DATA: 04/06/2019 | NUM: 104
verif_data.loc[2272, 'text'] = 'CONCEDER aposentadoria voluntaria ao servidor GIVALDO ANTONIO BATISTA DA CUNHA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 990-3, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 2299 | NOME: CELINA PEREIRA DE SOUZA | DATA: 06/06/2019 | NUM: 106
verif_data.loc[2299, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora CELINA PEREIRA DE SOUZA, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula no 1042-1, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 1o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 2393 | NOME: ELIZABETH DAS GRACAS CALDEIRA BRANT OLIVEIRA | DATA: 11/06/2019 | NUM: 109
verif_data.loc[2393, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora ELIZABETH DAS GRACAS CALDEIRA BRANT OLIVEIRA, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula no 757-9, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 1o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei 1.864/98.'

# Fix row 2444 | NOME: ROSILAINY DA FONSECA SIQUEIRA | DATA: 24/06/2019 | NUM: 116
verif_data.loc[2444, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora ROSILAINY DA FONSECA SIQUEIRA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 1006-5, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 2574 | NOME: PEDRO JOSE MARTINS SALGADO | DATA: 01/07/2019 | NUM: 45
verif_data.loc[2574, 'text'] = 'CONCEDER aposentadoria voluntaria ao servidor PEDRO JOSE MARTINS SALGADO, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula 932-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 3049 | NOME: ULISSES APARECIDO RIBEIRO | DATA: 08/08/2019 | NUM: 149
verif_data.loc[3049, 'text'] = 'CONCEDER aposentadoria voluntaria ao servidor ULISSES APARECIDO RIBEIRO, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula 964-4, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 3070 | NOME: HENRIQUE MATTEI | DATA: 16/08/2019 | NUM: 155
verif_data.loc[3070, 'text'] = 'CONCEDER aposentadoria voluntaria ao servidor HENRIQUE MATTEI, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula no 526-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 3853 | NOME: LUZEMAR RODRIGUES DOS SANTOS | DATA: 03/04/2018 | NUM: 63
verif_data.loc[3853, 'text'] = 'CONCEDER nos termos dos incisos I, II, III e paragrafo unico, artigo 3o, da Emenda Constitucional no 47, de 5 de julho de 2005, APOSENTADORIA VOLUNTARIA ao servidor LUZEMAR RODRIGUES DOS SANTOS, matricula no 93.544-1, Tecnico de Atividades Rodoviarias, Classe Especial, Padrao III, Referencia TR-S3, do Quadro de Pessoal do Departamento de Estradas de Rodagem do Distrito Federal. Processo no 00113-00008272/2018-27.'

# Fix row 4023 | NOME: TELMA FONTINELE QUEIROZ | DATA: 16/10/2018 | NUM: 197
verif_data.loc[4023, 'text'] = 'Conceder aposentadoria voluntaria a servidora TELMA FONTINELE QUEIROZ, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula 1029-4, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4300 | NOME: NAGELA CRISTIEN DO MONT | DATA: 20/11/2018 | NUM: 220
verif_data.loc[4300, 'text'] = 'CONCEDER aposentadoria a NAGELA CRISTIEN DO MONT, matricula no 43.492-2, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 1o, inciso II, alinea \"b\", da Lei Complementar no 51, de 20 de dezembro de 1985, com redacao dada pela Lei Complementar no 144, de 15 de maio de 2014, combinado com o artigo 40,  3o e 4o da Constituicao da Republica Federativa do Brasil, com redacao dada pela Emenda Constitucional no 20/1998, e com os artigos 3o e 7o da Emenda Constitucional no 41/2003.'

# Fix row 4351 | NOME: MARIO NUNES ATAIDES | DATA: 03/12/2018 | NUM: 228
verif_data.loc[4351, 'text'] = 'Conceder aposentadoria voluntaria ao servidor MARIO NUNES ATAIDES, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 901-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico, da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4352 | NOME: PAULO CEZAR CARNEIRO | DATA: 03/12/2018 | NUM: 228
verif_data.loc[4352, 'text'] = 'Conceder aposentadoria voluntaria ao servidor PAULO CEZAR CARNEIRO, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula 316-6, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, Paragrafo unico, da Lei no 1.864/98.'

# Fix row 4353 | NOME: GEUSA SANTANA DA SILVA | DATA: 03/12/2018 | NUM: 228
verif_data.loc[4353, 'text'] = 'Conceder aposentadoria voluntaria a servidora GEUSA SANTANA DA SILVA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 1122-3, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 1o e 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, e com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4368 | NOME: FRANCISCO SOARES DA SILVA | DATA: 17/12/2018 | NUM: 238
verif_data.loc[4368, 'text'] = 'Conceder aposentadoria voluntaria ao servidor FRANCISCO SOARES DA SILVA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula no 781-1, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 4383 | NOME: GIRLENE MOREIRA DA SILVA | DATA: 03/12/2018 | NUM: 228
verif_data.loc[4383, 'text'] = 'Conceder aposentadoria voluntaria a servidora GIRLENE MOREIRA DA SILVA, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula 790-1, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no art. 7o da Lei no 1.004/96, c/c com o art. 4o da Lei no 1.141/96, com o art. 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4388 | NOME: FRANCISCO JOSCELY TEIXEIRA ALBUQUERQUE | DATA: 03/12/2018 | NUM: 228
verif_data.loc[4388, 'text'] = 'Conceder aposentadoria voluntaria ao servidor FRANCISCO JOSCELY TEIXEIRA ALBUQUERQUE, Auditor de Controle Externo, Classe Especial, Padrao 63, matricula no 521-5, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

# Fix row 4400 | NOME: LUCIVANE DAYRELL LAGES NISHIMURA | DATA: 04/12/2018 | NUM: 81
verif_data.loc[4400, 'text'] = 'CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, bem como as vantagens do art. 5 da Lei 4.584/2011, a LUCIVA N E DAYRELL LAGES NISHIMURA, matricula no 134.525-7, na Carreira de Assistencia Pu'

# Fix row 4468 | NOME: LEILA MAGDA DE MELO | DATA: 27/12/2018 | NUM: 245
verif_data.loc[4468, 'text'] = 'CONCEDER aposentadoria por invalidez permanente a servidora LEILA MAGDA DE MELO, Tecnica de Administracao Publica, Classe Especial, Padrao 44, matricula no 1103-7, do Quadro de Pessoal dos Servicos Auxiliares deste Tribunal, com fundamento no artigo 40,  1o, inciso I, in fine, da CRFB, na redacao dada pela Emenda Constitucional no 41/03, combinado com o artigo 6o-A da Emenda Constitucional no 41/03, incluido pela Emenda Constitucional no 70/12, e o art. 18,  5o, da Lei Complementar no 769/08, com a vantagem pessoal prevista nos artigos 1o e 7o da Lei no 1.004/96, combinados com o artigo 4o da Lei no 1.141/96, e com o artigo 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 4796 | NOME: RACHEL BELLO AGUIAR DE LIMA | DATA: 27/07/2018 | NUM: 142
verif_data.loc[4796, 'text'] = 'CONCEDER APOSENTADORIA, nos termos do artigo 3o, incisos I, II, III, paragrafo unico da Emenda Constitucional no 47/2005, combinados com artigo 44 da Lei Complementar no 769, de 30/06/2008, a RACHEL BELLO AGUIAR DE LIMA, matricula no 127.919-X, na Carreira de Cirurgiao-Dentista, no Cargo de Cirurgiao-Dentista Odontologo, Classe Especial, Padrao IV, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: NUCE/GPCR. Processo: no 00060-00054308/2018-99.'

# Fix row 5002 | NOME: WILLIAM DA COSTA | DATA: 13/08/2018 | NUM: 153
verif_data.loc[5002, 'text'] = 'CONCEDER aposentadoria voluntaria ao servidor WILLIAM DA COSTA, Tecnico de Administracao Publica, Classe Especial, Padrao 44, matricula 975-0, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05, com a vantagem pessoal prevista no artigo 7o da Lei no 1.004/96, combinado com os artigos 4o da Lei no 1.141/96 e 4o, paragrafo unico, da Lei no 1.864/98.'

# Fix row 5030 | NOME: KLEYTAMAR DE ARAUJO | DATA: 17/08/2018 | NUM: 157
verif_data.loc[5030, 'text'] = 'AP O S E N TA R a servidora KLEYTAMAR DE ARAUJO, matricula no 57.594-1, no cargo efetivo de Agente de Policia, Classe Especial, da Carreira de Policia Civil do Distrito Federal, nos termos do artigo 40o, paragrafo 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41/2003, c/c artigo 6o-A da Emenda Cons- titucional no 41/2003, incluido pela Emenda Constitucional no 70/2012 e o artigo 186, inciso I, da Lei no 8.112/90.'

# Fix row 5142 | NOME: RAIMUNDA MENDES BRITO | DATA: 30/08/2018 | NUM: 166
verif_data.loc[5142, 'text'] = 'CONCEDER APOSENTADORIA, nos termos do artigo 2o, incisos I, II e III, alineas \"a\" e \"b\",  1o, inciso II, e  6o da Emenda Constitucional no 41/2003, e artigos 46 e 51 da Lei Complementar no 769/2008, a RAIMUNDA MENDES BRITO, matricula no 159.287-4, na Carreira Medica, no Cargo de MEDICO - PEDIATRIA, Segunda Classe, Padrao V, do Quadro de Pessoal da Secretaria de Estado de Saude do Distrito Federal. Lotacao: SRSNO. Processo: no 00060-00034069/2018-51.'

# Fix row 5170 | NOME: OLGA LEITE DA SILVA | DATA: 03/09/2018 | NUM: 168
verif_data.loc[5170, 'text'] = 'CONCEDER aposentadoria voluntaria a servidora OLGA LEITE DA SILVA, Auxiliar de Administracao Publica, Classe Especial, Padrao 25, matricula no 1109-6, do Quadro de Pessoal dos Servicos Auxiliares desta Corte, fundamentada no art. 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional no 47/05.'

In [139]:
cnt = 0
for idx in verif_data.index:
    if not verif_data.loc[idx, 'text']:
        cnt += 1
print("Anotacoes de aposentadoria")
print("Total:", len(verif_data))
print("Encontradas:", len(verif_data)-cnt)
print("Nao encontradas:", cnt)

Anotacoes de aposentadoria
Total: 5516
Encontradas: 5516
Nao encontradas: 0


## PARTE 5 - Identificando trechos repetidos

Corrigindo erro em que dois atos separados de aposentadoria de uma mesma pessoa em um mesmo DODF pegam o mesmo trecho de texto

In [140]:
multi_names = []
for idx in verif_data.index:
    if len(verif_data[verif_data['NOME_ATO'] == verif_data.loc[idx, 'NOME_ATO']]) >= 2:
        if verif_data.loc[idx, 'NOME_ATO'] not in multi_names:
            multi_names.append(verif_data.loc[idx, 'NOME_ATO'])
len(multi_names)

162

In [141]:
tri_names = []
duo_names = []
for name in multi_names:
    aux = verif_data[verif_data['NOME_ATO'] == name]
    if len(aux) == 2:
        if aux['text'][aux.index[0]] == aux['text'][aux.index[1]]:
            if aux['NUM_DODF'][aux.index[0]] == aux['NUM_DODF'][aux.index[1]]:
                duo_names.append(name)
    else:
        tri_names.append(name)
print(f"Numero de nomes que aparecem em dois atos de aposentadoria no mesmo DODF:{len(duo_names)}")
print(f"Numero de nomes que aparecem em tres atos de aposentadoria:{len(tri_names)}")

Numero de nomes que aparecem em dois atos de aposentadoria no mesmo DODF:16
Numero de nomes que aparecem em tres atos de aposentadoria:20


#### Pegando o segundo ato de aposentadoria quando uma pessoa é citada mais de uma vez no mesmo DODF

In [142]:
# Fix row 3199
verif_data.loc[3199, 'text'] = "APOSENTAR SOLANGE DA CRUZ SANTOS, matricula 39.734-2, no Cargo de Professor de Educacao Basica, Padrao 22, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00040-00013974/2019-22."

# Fix row 4217
verif_data.loc[4217, 'text'] = "CONCEDER APOSENTADORIA a NANCI CALAZANS DA SILVA, matricula 35.763-4, no Ca rg o de Professor de Educacao Basica, Padrao 25, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00061320/2018-85."

# Fix row 4221
verif_data.loc[4221, 'text'] = "CONCEDER APOSENTADORIA a GLAUCIA MARIA DE SOUZA, matricula 21.065-X, no Carg o de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00061329/2018-96."

# Fix row 4222
verif_data.loc[4222, 'text'] = "CONCEDER APOSENTADORIA a CLEONICE ANTONIA MARTINS FERREIRA, matricula 67.188- 6, no Cargo de Agente de Gestao Educacional/Conservacao e Limpeza, Nivel 10, Padrao 03, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional n 47 de 06 de julho de 2005. Processo 00080-00121333/2018- 11 ."

# Fix row 4223
verif_data.loc[4223, 'text'] = "CONCEDER APOSENTADORIA a CLAUDIA PATRICIA DE OLIVEIRA LIMA, matricula 49.700- 2, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00027167/2017-86."

# Fix row 4224
verif_data.loc[4224, 'text'] = "APOSENTAR RITA DE CASSIA LORETTI WERNECK PINTO, matricula 38.203-5, no Cargo de Professor de Educacao Basica, Padrao 22, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00080-00053467/2018-00."

# Fix row 4225
verif_data.loc[4225, 'text'] = "APOSENTAR NATHALIE DE FREITAS RODRIGUES, matricula 41.828-5, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00410-00010284/2018-67."

# Fix row 4226
verif_data.loc[4226, 'text'] = "APOSENTAR LILIAN NUBIA CAFE MELO ISSA, matricula 47.437-1, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, in fine, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012, e o artigo 18,  5o, da Lei Complementar no 769, de 01 de julho de 2008. Processo 00410-00011074/2018-96."

# Fix row 4233
verif_data.loc[4233, 'text'] = "CONCEDER APOSENTADORIA a ROSALIA RIBEIRO DE OLIVEIRA, matricula 62.283-4, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00066022/2018-81."

# Fix row 4248
verif_data.loc[4248, 'text'] = "CONCEDER APOSENTADORIA a ANDREA SILVA DE CASTRO, matricula 45.004-9, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00001545/2018-82."

# Fix row 4256
verif_data.loc[4256, 'text'] = "CONCEDER APOSENTADORIA a DIVINA DE JESUS PEIXOTO, matricula 46.465-1, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa III, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 3o, incisos I, II e III, e paragrafo unico da Emenda Constitucional n 47 de 06 de julho de 2005. Processo 00080-00114729/2018-10."

# Fix row 4449
verif_data.loc[4449, 'text'] = "APOSENTAR CRISTINA VASCONCELOS DE PAULO, matricula 205.726-3, no Cargo de Professor de Educacao Basica, Padrao 14, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do artigo 40, 1o, inciso I, da Constituicao da Republica Federativa do Brasil, na redacao dada pela Emenda Constitucional no 41, de 31 de dezembro de 2003, combinado com o artigo 6o-A da Emenda Constitucional no 41, de 31 de dezembro de 2003, incluido pela Emenda Constitucional no 70, de 29 de marco de 2012. Processo 00410-00014667/2018-12."

# Fix row 4220
verif_data.loc[4220, 'text'] = "CONCEDER APOSENTADORIA a JACIARA ALVES VAZ, matricula 64.931-7, no Cargo de Professor de Educacao Basica, Padrao 25, Etapa IV, do Quadro de Pessoal do Distrito Federal, nos termos do Artigo 6o da Emenda Constitucional no 41, de 31 de dezembro de 2003, artigo 40,  5o, da Constituicao da Republica Federativa do Brasil, na redacao da Emenda Constitucional no 20, de 16 de dezembro de 1998, e artigo 2o da Emenda Constitucional no 47, de 06 de julho de 2005. Processo 00080-00085736/2018-99."


## Salvando os textos encontrados

Salva os trechos de texto identificados juntamente as entidades anotadas em arquivo .csv

In [143]:
verif_data.to_csv('verif_dataV1.csv', index = False)